Resources:
https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/TAPAS/Fine_tuning_TapasForQuestionAnswering_on_SQA.ipynb#scrollTo=QP4AHMxFujhV


In [20]:
#tokenizer = TapasTokenizer.from_pretrained('google/tapas-base-finetuned-wtq')
#tokenizer = TapasTokenizer.from_pretrained('google/tapas-large-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base-finetuned-wtq')
#model = TapasForQuestionAnswering.from_pretrained('google/tapas-large-finetuned-wtq')

from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("google/tapas-large-finetuned-sqa")
#model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-large-finetuned-sqa")

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [21]:
import pandas as pd
import datetime

In [22]:
#from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
import torch

#model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(1024, 1024)
      (token_type_embeddings_0): Embedding(3, 1024)
      (token_type_embeddings_1): Embedding(256, 1024)
      (token_type_embeddings_2): Embedding(256, 1024)
      (token_type_embeddings_3): Embedding(2, 1024)
      (token_type_embeddings_4): Embedding(256, 1024)
      (token_type_embeddings_5): Embedding(256, 1024)
      (token_type_embeddings_6): Embedding(10, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0): TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024

In [23]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=10)).isoformat()
#Get count of incidents to determine limit to retrieve
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)
# Use record to write query call to API to get all incidents needed/bypass API default of 1000
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +str(periodstart)+"%27%20AND%20%27"
           +str(periodend)+"%27&$limit="+str(maxNumber))
cfs_data=pd.read_json(query_str)
print(cfs_data.shape)
cfs_data.sample(5)

The number of incidents in the set is 7638
(7638, 34)


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,transport_dttm,hospital_dttm
1519,220221156,T01,22010419,Elevator / Escalator Rescue,2022-01-22T00:00:00.000,2022-01-22T00:00:00.000,2022-01-22T11:06:14.000,2022-01-22T11:08:06.000,2022-01-22T11:08:21.000,2022-01-22T11:12:05.000,...,1,TRUCK,1,3,6,South of Market,220221156-T01,"{'type': 'Point', 'coordinates': [-122.4113180...",NaN,NaN
3377,220252094,B02,22011896,Alarms,2022-01-25T00:00:00.000,2022-01-25T00:00:00.000,2022-01-25T15:15:34.000,2022-01-25T15:17:30.000,2022-01-25T15:18:01.000,2022-01-25T15:19:17.000,...,1,CHIEF,2,2,6,Tenderloin,220252094-B02,"{'type': 'Point', 'coordinates': [-122.4169793...",NaN,NaN
470,220212371,T12,22010128,Alarms,2022-01-21T00:00:00.000,2022-01-21T00:00:00.000,2022-01-21T16:36:41.000,2022-01-21T16:37:32.000,2022-01-21T16:38:34.000,2022-01-21T16:40:01.000,...,1,TRUCK,3,5,5,Haight Ashbury,220212371-T12,"{'type': 'Point', 'coordinates': [-122.4444860...",NaN,NaN
3443,220250363,T16,22011679,Outside Fire,2022-01-25T00:00:00.000,2022-01-24T00:00:00.000,2022-01-25T06:02:40.000,2022-01-25T06:03:53.000,2022-01-25T06:04:23.000,2022-01-25T06:06:55.000,...,1,TRUCK,3,1,3,Nob Hill,220250363-T16,"{'type': 'Point', 'coordinates': [-122.4104061...",NaN,NaN
6364,220281563,63,22013246,Medical Incident,2022-01-28T00:00:00.000,2022-01-28T00:00:00.000,2022-01-28T12:57:05.000,2022-01-28T12:57:55.000,2022-01-28T12:58:18.000,2022-01-28T12:58:52.000,...,1,MEDIC,2,2,9,Mission,220281563-63,"{'type': 'Point', 'coordinates': [-122.4154689...",2022-01-28T13:17:04.000,2022-01-28T13:28:47.000


In [34]:
cfs_data=cfs_data.dropna()
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(10)
cfs_data_tapas=cfs_small[['call_type','neighborhoods_analysis_boundaries']]
#cfs_data_tapas=cfs_small[['unit_type','call_type','neighborhoods_analysis_boundaries']]

In [35]:
#cfs_data_tapas=cfs_data_tapas.rename(columns={"unit_type":"unit","call_type":"incident","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_data_tapas=cfs_data_tapas.rename(columns={"call_type":"incident","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_data_tapas

,incident,neighbourhood
5,Medical Incident,Sunset/Parkside
6,Medical Incident,Western Addition
12,Medical Incident,Outer Richmond
25,Medical Incident,Sunset/Parkside
26,Medical Incident,Marina
34,Medical Incident,McLaren Park
42,Medical Incident,Tenderloin
43,Medical Incident,Mission
47,Medical Incident,Nob Hill
49,Medical Incident,Bayview Hunters Point


In [36]:
queries = ["What unit types are there?", "How many incidents are in Tenderloin?"]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', truncation=True, return_tensors="pt")

logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

IndexError: iloc cannot enlarge its target object

In [ ]:
encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.answer_coordinates, answer_text=item.answer_text,
                     truncation=True, padding="max_length", return_tensors="pt")
encoding.keys()

In [ ]:
data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 
        'Age': ["56", "45", "59"],
        'Number of movies': ["87", "53", "69"],
        'Date of birth': ["7 february 1967", "10 june 1996", "28 november 1967"]}
queries = ["How many movies has George Clooney played in?", "How old is he?", "What's his date of birth?"]

table = pd.DataFrame.from_dict(data)

inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
logits = compute_prediction_sequence(model, inputs, device)

In [ ]:
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(500)
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

queries = ["When was the most recent medical incident?", "How many incidents are in Tenderloin?"]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(500)
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

#cfs_data_tapas=cfs_small[['call_number','call_date','call_type','unit_type','neighborhoods_analysis_boundaries']]
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

cfs_data_tapas

In [ ]:
queries = ["How many medical incidents?","What type of calls are there?" ]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

In [ ]:
logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering
#config = TapasConfig('google-base-finetuned-wikisql-supervised')
#model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base')

In [ ]:
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
config = TapasConfig.from_pretrained('google/tapas-base-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
# the base sized model with WikiSQL configuration
#config = TapasConfig('google-base-finetuned-wikisql-supervised')
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")


In [ ]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.sample(100)
#cfs_data_tapas=cfs_small[['call_type','neighborhoods_analysis_boundaries']]
#cfs_data_tapas=cfs_data_tapas.rename(columns={"call_type":"incident type","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_small

In [ ]:
list_of_list = [[]]
list_of_list[0] = list(cfs_small.columns)
list_of_list.extend(cfs_small.values.tolist())

In [ ]:
result = predict(list_of_list, ["How many medical incidents?","Which neighbour are there?"])

In [ ]:
queries = ["How many medical incidents?","Which neighbour are there?"]
inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

In [ ]:
model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)